In [0]:
#TO run the mask rcnn ,
#We need to have the local repository and the associated weights of it 
#We need to download all the required packages 
#Using conda we will be installing all required packages 

In [0]:
!git clone https://github.com/matterport/Mask_RCNN

In [0]:
!pip install tqdm

In [0]:
!git clone https://github.com/EscVM/OIDv4_ToolKit.git

In [0]:
cd Mask_RCNN

In [0]:
!wget https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5

In [0]:
!conda update conda --yes

In [0]:
!conda install numpy --yes

In [0]:
!conda install tensorflow=1.13.1 --yes

In [0]:
!conda install -c menpo opencv --yes

In [0]:
!conda install -c conda-forge pycocotools --yes

In [0]:
!conda install -c conda-forge imgaug --yes

In [0]:
!conda install -c conda-forge keras --yes

In [0]:
#We will be freezing all the required and installed packages to run the mask rcnn project 
#So for the next time when required to run this project , installing from the require.txt is sufficient 

'''
##For this project ,
1. We had to install the packages and import them 
2. Load the weights to the model
3. Load all the config parameters to the model
4. Run the model on the images 

'''

In [0]:
!pip freeze > reqire.txt

In [0]:
cd Mask_RCNN/

In [0]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import glob
import matplotlib
import matplotlib.pyplot as plt
import zipfile
from PIL import Image, ImageDraw, ImageFont
import colorsys
import tqdm

# Root directory of the project
ROOT_DIR = os.path.abspath(".")

import warnings
warnings.filterwarnings("ignore")
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco


%matplotlib inline

In [0]:
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join('.', "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [0]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

In [0]:
model = modellib.MaskRCNN(mode="inference", model_dir='mask_rcnn_coco.hy', config=config)

# Load weights trained on MS-COCO
model.load_weights('mask_rcnn_coco.h5', by_name=True)

In [0]:
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [0]:
!wget https://github.com/JotJunior/PHP-Boleto-ZF2/blob/master/public/assets/fonts/arial.ttf

In [0]:
'''
We need to save the output of the model with assigning different colors to different objects , the below function was later added to take in the mask values
and save them 

We also had to label them with a font, which is why we are downloading Arial.tff file to use it 
'''
def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors




def save_image(image, image_name, boxes, masks, class_ids, scores, class_names, filter_classs_names=None,
               scores_thresh=0.1, save_dir=None, mode=0):
    """
        image: image array
        image_name: image name
        boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
        masks: [num_instances, height, width]
        class_ids: [num_instances]
        scores: confidence scores for each box
        class_names: list of class names of the dataset
        filter_classs_names: (optional) list of class names we want to draw
        scores_thresh: (optional) threshold of confidence scores
        save_dir: (optional) the path to store image
        mode: (optional) select the result which you want
                mode = 0 , save image with bbox,class_name,score and mask;
                mode = 1 , save image with bbox,class_name and score;
                mode = 2 , save image with class_name,score and mask;
                mode = 3 , save mask with black background;
    """
    mode_list = [0, 1, 2, 3]
    assert mode in mode_list, "mode's value should in mode_list %s" % str(mode_list)

    if save_dir is None:
        save_dir = os.path.join(os.getcwd(), "output")
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

    useful_mask_indices = []

    N = boxes.shape[0]
    if not N:
        print("\n*** No instances in image %s to draw *** \n" % (image_name))
        return
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    for i in range(N):
        # filter
        class_id = class_ids[i]
        score = scores[i] if scores is not None else None
        label = class_names[class_id]
        if (filter_classs_names is not None) and (label not in filter_classs_names):
            continue

        if not np.any(boxes[i]):
            # Skip this instance. Has no bbox. Likely lost in image cropping.
            continue

        useful_mask_indices.append(i)

    if len(useful_mask_indices) == 0:
        print("\n*** No instances in image %s to draw *** \n" % (image_name))
        return

    colors = random_colors(len(useful_mask_indices))

    if mode != 3:
        masked_image = image.astype(np.uint8).copy()
    else:
        masked_image = np.zeros(image.shape).astype(np.uint8)

    if mode != 1:
        for index, value in enumerate(useful_mask_indices):
            masked_image = visualize.apply_mask(masked_image, masks[:, :, value], colors[index])

    masked_image = Image.fromarray(masked_image)

    if mode == 3:
        masked_image.save(os.path.join(save_dir, '%s.jpg' % (image_name)))
        return

    draw = ImageDraw.Draw(masked_image)
    colors = np.array(colors).astype(int) * 255

    for index, value in enumerate(useful_mask_indices):
        class_id = class_ids[value]
        score = scores[value]
        label = class_names[class_id]

        y1, x1, y2, x2 = boxes[value]
        if mode != 2:
            color = tuple(colors[index])
            draw.rectangle((x1, y1, x2, y2), outline=color)

        # Label
        print(os.getcwd())
        font = ImageFont.truetype('Arial.ttf', 15)
        draw.text((x1, y1), "%s %f" % (label, score), (255, 255, 255), font)
    print("saving here ",os.path.join(save_dir, '%s.jpg' % (image_name)))
    masked_image.save(os.path.join(save_dir, '%s.jpg' % (image_name)))

In [0]:
"""
1st on opencv where the frame rate has been corrected
2nd is kaggle images
3rd is openimag

"""

In [0]:
'''
We load in the video named market.mp4 to split them into frames and then load them into the model
this is for opencv
'''

import cv2
vidcap = cv2.VideoCapture('MARKET.mp4')
success,image1 = vidcap.read()
count = 0
frame_rate = 10
prev = 0

while success:
 

    time_elapsed = time.time() - prev
    if time_elapsed > 1./frame_rate:
        prev = time.time()
        cv2.imwrite("frame%d.jpg" % count, image1)     # save frame as JPEG file      
        success,image = vidcap.read()
        print('Read a new frame: ', success)
        count += 1
        results = model.detect([image], verbose=1)
        # Visualize results
        r = results[0]
        save_image(image,"{}".format(count), r['rois'], r['masks'], r['class_ids'],r['scores'], class_names)

In [0]:
''''
We will be using glob to take all the jpg images and then loading them into the model to get required output
This is for kaggle dataset , openimages dataset  and cocodataset 
'''

for i in glob.glob("../kaggle/*.jpg"):
    try:
        image = skimage.io.imread(i)
        x='output/'+i.split("/")[2].split(".")[0]+'.jpg'
        if x  not in glob.glob("output/*.jpg"):
            results = model.detect([image], verbose=1)
            r = results[0]
            save_image(image,"{}".format(i.split("/")[2].split(".")[0]), r['rois'], r['masks'], r['class_ids'],r['scores'], class_names)
        else:
                print("image already done \n")
    except:
            print(i)

In [0]:
!python main.py downloader --classes Apple Orange --type_csv train --limit 10 --yes

In [0]:
cd OIDv4_ToolKit/OID/Dataset/train/

In [0]:
for i in glob.glob("/Apple/*.jpg"):
    try:
        image = skimage.io.imread(i)
        x='output/'+i.split("/")[2].split(".")[0]+'.jpg'
        if x  not in glob.glob("output/*.jpg"):
            results = model.detect([image], verbose=1)
            r = results[0]
            save_image(image,"{}".format(i.split("/")[2].split(".")[0]), r['rois'], r['masks'], r['class_ids'],r['scores'], class_names)
        else:
                print("image already done \n")
    except:
            print(i)

In [0]:
'''
Since sagemaker wont allow us to download multiple file together , we just zip the output images  and download the zip file s
'''

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        print(root)
        for file in files:
            ziph.write(os.path.join(root, file))

zipf = zipfile.ZipFile('Python.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('output', zipf)
zipf.close()

In [0]:
#OIDV4 image clone
!git clone https://github.com/EscVM/OIDv4_ToolKit.git
!pip install tqdm
import tqdm

In [0]:
cd OIDv4_ToolKit/

In [0]:
#Extracting 400 images from OIDV4 of classes Apple and Orange
!python main.py downloader --classes Apple Orange --type_csv train --limit 200 --yes

In [0]:
cd OID/Dataset/train/